
# Binary‑Recursion Deep‑Dive 📊🔬  
This notebook reproduces and **extends** the three sanity checks:

1. **Peak‑resolution run** with adaptive binning (1 × 10⁻⁷) and KDE.  
2. **Feigenbaum δ** via a simple control‑parameter sweep (twist phase).  
3. **α/π proximity Z‑score** using a Monte‑shuffle test.  

> **How to use:** Run top‑to‑bottom in Colab (no extra installs).  
> Built for Python 3.10 +, NumPy, Matplotlib, SciPy.


In [ ]:

import numpy as np, matplotlib.pyplot as plt, math, random, itertools, collections
from mpmath import mp, mpf
from scipy.stats import gaussian_kde, zscore
mp.dps = 50

phi = (1 + mp.sqrt(5)) / 2
pi  = mp.pi
alpha_inv = mp.mpf('137.035999084')  # CODATA 2022
alpha = 1 / alpha_inv
print("phi =", phi)
print("pi  =", pi)
print("alpha_inv =", alpha_inv)


In [ ]:

def binary_squared(max_len):
    """Return ndarray of squared binary‑string values up to `max_len`."""
    vals = []
    for n in range(1, max_len+1):
        denom = 2**n
        vals.extend([(i/denom)**2 for i in range(denom)])
    return np.array(vals)


In [ ]:

def peak_from_hist(data, bin_width=1e-7):
    bins = np.arange(0, 1+bin_width, bin_width)
    counts, edges = np.histogram(data, bins=bins)
    idx = counts.argmax()
    peak_x = 0.5*(edges[idx]+edges[idx+1])
    peak_h = counts[idx]
    return peak_x, peak_h, edges[:-1], counts

def kde_peak(data, bw=1e-7):
    kde = gaussian_kde(data, bw_method=0.3)
    xs = np.linspace(0, max(data), 5000)
    ys = kde(xs)
    idx = ys.argmax()
    return xs[idx], ys[idx], xs, ys


In [ ]:

def twist_map(x, phase):
    return (x**2 + phase) % 1.0

def bifurcation_diagram(phases, iters=600, drop=100, samples=200):
    xs, ps = [], []
    for p in phases:
        x = 0.5
        for _ in range(drop):
            x = twist_map(x, p)
        for _ in range(samples):
            x = twist_map(x, p)
            xs.append(x)
            ps.append(p)
    return np.array(ps), np.array(xs)

phases = np.linspace(0.0, 0.25, 300)   # quarter‑phase sweep
P, X = bifurcation_diagram(phases)

plt.figure(figsize=(6,4))
plt.scatter(P, X, s=0.1, alpha=0.5)
plt.title('Bifurcation diagram (phase vs x)')
plt.xlabel('phase'); plt.ylabel('x')
plt.show()


In [ ]:

def period_doubling_params(phases, tol=1e-3):
    """Return phase values where number of clusters doubles."""
    params = []
    last_count = None
    window = 200
    for p in phases:
        x = 0.1234
        seen = set()
        for _ in range(window):
            x = twist_map(x, p)
            seen.add(round(x,6))
        count = len(seen)
        if last_count and count >= 2*last_count - tol and abs(count - 2*last_count) <= 2:
            params.append(p)
        last_count = count
    return params

pd_params = period_doubling_params(phases)
print("Period‑doubling phase values:", pd_params[:5])

deltas = []
for i in range(2, len(pd_params)):
    deltas.append((pd_params[i-1]-pd_params[i-2])/(pd_params[i]-pd_params[i-1]))
if deltas:
    print("Feigenbaum Δ estimates:", deltas, "mean≈", np.mean(deltas))


In [ ]:

def alpha_pi_grid(k_max=100):
    return np.array([k*alpha/mp.pi for k in range(1, k_max+1)], dtype=float)

def spike_alpha_distance(spike_x, grid):
    return np.min(np.abs(grid - spike_x))

grid = alpha_pi_grid(500)
obs_dist = spike_alpha_distance(kpx, grid)
print("Observed spike distance to α/π grid:", obs_dist)

# Monte shuffle
random.seed(42)
vals_full = binary_squared(23)
dists = []
for _ in range(500):
    samp_peak,_,_,_ = peak_from_hist(random.sample(list(vals_full), len(vals_full)))
    dists.append(spike_alpha_distance(samp_peak, grid))
z = (obs_dist - np.mean(dists)) / np.std(dists)
print(f'α/π proximity Z‑score = {z:.2f}σ  (higher ⇒ less likely by chance)')
